In [0]:
import bs4 as bs
import urllib.request
import pandas as pd
from time import sleep
from lxml import html  
import requests
from time import sleep
import json
import argparse
from collections import OrderedDict
import lxml.html as lh
import pandas as pd

In [0]:
def getSummaryData(url):
  response = requests.get(url, verify=False)
  print ("Parsing %s"%(url))
  sleep(4)
  parser = html.fromstring(response.text)
  summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
  summary_data = OrderedDict()
  try:
    for table_data in summary_table:
      raw_table_key = table_data.xpath('.//td[contains(@class,"C($primaryColor)")]//text()')
      raw_table_value = table_data.xpath('.//td[contains(@class,"Ta(end)")]//text()')
      table_key = ''.join(raw_table_key).strip()
      table_value = ''.join(raw_table_value).strip()
      summary_data.update({table_key:table_value})
    return summary_data
  
  except:
    print ("Failed to parse json response")
    return {"error":"Failed to parse json response"}
  
  

In [0]:
def getFinanceData(url):
  response = requests.get(url, verify=False)
  print ("Parsing %s"%(url))
  sleep(4)
  parser = html.fromstring(response.text)
  summary_table = parser.xpath('//div[contains(@data-test,"fin-row")]')
  print(len(summary_table))
  summary_data = pd.DataFrame()

  for table_data in summary_table:
    raw_table_key = table_data.xpath('.//span[contains(@class,"Va(m)")]//text()')

    raw_table_value = table_data.xpath('.//span[not(contains(@class,"Va(m)"))]//text()')

    print('table_key', raw_table_key)
    print('table_value', raw_table_value)
    
  return summary_data
    
# empty cells, heirarchical cells
  


In [4]:
financeUrl = "https://finance.yahoo.com/quote/TSLA/financials?p=TSLA"
scrappedData = getFinanceData(financeUrl)
print(scrappedData)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Parsing https://finance.yahoo.com/quote/TSLA/financials?p=TSLA
22
table_key ['Total Revenue']
table_value ['24,941,426', '21,461,268', '11,758,751', '7,000,132', '4,046,024']
table_key ['Cost of Revenue']
table_value ['20,488,072', '17,419,247', '9,536,264', '5,400,875', '3,122,521']
table_key ['Gross Profit']
table_value ['4,453,354', '4,042,021', '2,222,487', '1,599,257', '923,503']
table_key ['Operating Expenses', 'Research Development', 'Selling General and Administrative', 'Total Operating Expenses']
table_value ['1,371,217', '1,460,370', '1,378,073', '834,408', '717,900', '2,748,518', '2,834,491', '2,476,500', '1,432,189', '922,232', '4,119,735', '4,294,861', '3,854,573', '2,266,597', '1,640,132']
table_key ['Research Development']
table_value ['1,371,217', '1,460,370', '1,378,073', '834,408', '717,900']
table_key ['Selling General and Administrative']
table_value ['2,748,518', '2,834,491', '2,476,500', '1,432,189', '922,232']
table_key ['Total Operating Expenses']
table_value ['

In [0]:
def parse_url(url):
  response = requests.get(url)
  soup = bs.BeautifulSoup(response.text, 'lxml')
  
  listdf = []
  
  for table in soup.find_all('table'):
    listdf.append(parse_html_table(table))
  
  return listdf
    
def parse_html_table(table):
  n_columns = 0
  n_rows=0
  column_names = []

  # Find number of rows and columns
  # we also find the column titles if we can
  for row in table.find_all('tr'):

      # Determine the number of rows in the table
      td_tags = row.find_all('td')
      if len(td_tags) > 0:
          n_rows+=1
          if n_columns == 0:
              # Set the number of columns for our table
              n_columns = len(td_tags)

      # Handle column names if we find them
      th_tags = row.find_all('th') 
      if len(th_tags) > 0 and len(column_names) == 0:
          for th in th_tags:
              column_names.append(th.get_text())
  
  # Safeguard on Column Titles
  if len(column_names) > 0 and len(column_names) != n_columns:
      raise Exception("Column titles do not match the number of columns")

  columns = column_names if len(column_names) > 0 else range(0,n_columns)
  df = pd.DataFrame(columns = columns,
                    index= range(0,n_rows))
  row_marker = 0
  for row in table.find_all('tr'):
      column_marker = 0
      columns = row.find_all('td')
      for column in columns:
          df.iat[row_marker,column_marker] = column.get_text()
          column_marker += 1
      if len(columns) > 0:
          row_marker += 1

  # Convert to float if possible
  for col in df:
      try:
          df[col] = df[col].astype(float)
      except ValueError:
          pass

  return df

In [6]:
from google.colab import drive
drive.mount('drive')
cmpList = ['TSLA', 'AAPL', 'NFLX']
#https://finance.yahoo.com/quote/%5EIXIC/components?p=%5EIXIC

items = ['', '/analysis']
urlPre = 'https://finance.yahoo.com/quote/'
urlSuf = '?p='

columns = None

#scrappedData = getSummaryData(summaryUrl)
# response = requests.get(financeUrl, verify=False)
# soup = bs.BeautifulSoup(response.text)
# print(soup.prettify())
final_df = pd.DataFrame()

for cmp in cmpList:
  cmp_df_list = []
  print('\nCompany: ', cmp)
  
  for item in items:
    url = urlPre + cmp + item + urlSuf + cmp
    dataframe_list = parse_url(url)
    
    if item is '':
      name = 'summary'
    else:
      name = item[1:len(item)]
    
    print(name, len(dataframe_list), 'tables')
    
    count = 1
    
    item_df = pd.DataFrame()
    
    for df in dataframe_list:
    
      # Summary (2), Analysis (6)
      
      if item is "":
        df = df.transpose()
        item_df = pd.concat([item_df, df], axis = 1, ignore_index = True)
        
        
        
      if item == '/analysis':
        df.set_index(df.columns[0], inplace=True)
        df = df.replace({pd.np.nan: None})
        df = df.unstack().to_frame().sort_index(level=1).T
        df.columns = df.columns.map("_".join)
        item_df = pd.concat([item_df, df], axis = 1)
     
    
    if item is "":
      new_header = item_df.iloc[0] #grab the first row for the header
#       print(item_df)
      item_df.columns = new_header #set the header row as the df header
#       print(item_df)
      item_df = item_df[1:] #take the data less the header row
#       print(item_df)
#       print(new_header)
      item_df.reset_index(inplace=True, drop=True)
     
      
    #print(item_df)
    print(name, item_df.shape[1], 'Columns')
    cmp_df_list.append(item_df)
    
#     filename = cmp + '_' + name + '.csv'
#     item_df.to_csv(filename, index = False)
#     !cp $filename drive/My\ Drive/Isenberg/
  
  
  
  cmp_df = pd.DataFrame()
  for df in cmp_df_list:
    cmp_df = pd.concat([cmp_df,df], axis = 1)
  
  cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
  
  if columns == None:
    columns = cmp_df.columns.tolist()
 
  cmp_df.sort_index(axis=1, inplace=True)

  final_df = pd.concat([final_df, cmp_df], ignore_index = True)

final_df['company'] = cmpList  
final_df = final_df.set_index('company')
print('Final Data: ',final_df.shape)

filename = 'YahooData.csv'
final_df.to_csv(filename)
!cp $filename drive/My\ Drive/Isenberg/

KeyboardInterrupt: ignored

In [0]:
# #listdf = parse_url("https://www.marketwatch.com/investing/stock/aapl/financials")
# listf = parse_url("https://old.nasdaq.com/symbol/aapl/financials?query=balance-sheet")
# for df in listdf:
#   print(df)

In [9]:
from google.colab import drive
drive.mount('drive')
cmpList = ['TSLA', 'AAPL', 'NFLX']
#https://finance.yahoo.com/quote/%5EIXIC/components?p=%5EIXIC

items = ['/financials']
urlPre = 'https://www.marketwatch.com/investing/stock/'

columns = None


final_df = pd.DataFrame()

for cmp in cmpList:
  cmp_df_list = []
  print('\nCompany: ', cmp)
  
  for item in items:
    url = urlPre + cmp + item
    dataframe_list = parse_url(url)
    
    if item is '':
      name = 'summary'
    else:
      name = item[1:len(item)]
    
    print(name, len(dataframe_list), 'tables')
    
    count = 1
    
    item_df = pd.DataFrame()
    
    for df in dataframe_list:
    
      if item is "":
        df = df.transpose()
        item_df = pd.concat([item_df, df], axis = 1, ignore_index = True)
        
        
        
      if item == '/financials':
        df.set_index(df.columns[0], inplace=True)
        df = df.replace({pd.np.nan: None})
        df = df.unstack().to_frame().sort_index(level=1).T
        df.columns = df.columns.map("_".join)
        item_df = pd.concat([item_df, df], axis = 1)
     
    
    if item is "":
      new_header = item_df.iloc[0] #grab the first row for the header
#       print(item_df)
      item_df.columns = new_header #set the header row as the df header
#       print(item_df)
      item_df = item_df[1:] #take the data less the header row
#       print(item_df)
#       print(new_header)
      item_df.reset_index(inplace=True, drop=True)
     
      
    #print(item_df)
    print(name, item_df.shape[1], 'Columns')
    cmp_df_list.append(item_df)
    
#     filename = cmp + '_' + name + '.csv'
#     item_df.to_csv(filename, index = False)
#     !cp $filename drive/My\ Drive/Isenberg/
  
  
  
  cmp_df = pd.DataFrame()
  for df in cmp_df_list:
    cmp_df = pd.concat([cmp_df,df], axis = 1)
  
  cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
  
  if columns == None:
    columns = cmp_df.columns.tolist()
 
  cmp_df.sort_index(axis=1, inplace=True)

  final_df = pd.concat([final_df, cmp_df], ignore_index = True)

final_df['company'] = cmpList  
final_df = final_df.set_index('company')
print('Final Data: ',final_df.shape)

filename = 'MarketWatchData.xlsx'
final_df.to_excel(filename)
!cp $filename drive/My\ Drive/Isenberg/

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).

Company:  TSLA
financials 2 tables
financials 342 Columns

Company:  AAPL
financials 2 tables
financials 342 Columns

Company:  NFLX
financials 2 tables
financials 342 Columns
Final Data:  (3, 342)
